<a href="https://colab.research.google.com/github/mab-rook/BVAVS/blob/master/newFacial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

%cd root
%cd /content/gdrive/MyDrive
%mkdir csefacerec
%cd csefacerec
%mkdir enrol
%cd enrol
%mkdir pickle
!pip install requests
!pip install face_recognition

In [ ]:
import requests
import base64
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt
import os
import pickle
import face_recognition
import numpy as np
import cv2
import os
import time
import shutil
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import face_recognition
import pickle
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db

while True:
      # Define the parent directory for saving images
      folder = "verification"
      if os.path.exists(folder):
          shutil.rmtree(folder)

      os.makedirs(folder)
      print("Directory '{}' created.".format(folder))

      # URL to fetch the single image for user ID = 1
      image_url = 'https://medcenter.com.ng/bvas/fetch_voter.php'

      # Base directory where the folder will be created
      base_dir = "voter"

      # Create the base directory if it doesn't exist
      if not os.path.exists(base_dir):
          os.makedirs(base_dir)

      # Define the path for the new folder
      folder_path = os.path.join(base_dir, '1')

      # Create the folder if it doesn't already exist
      if not os.path.exists(folder_path):
          os.makedirs(folder_path)
          print(f"Folder created for user ID: 1")

      # Fetch the image for the user ID = 1
      image_response = requests.get(image_url)

      # Check if the request was successful
      if image_response.status_code == 200:
          image_data = image_response.json()

          # Check if the response is successful
          if image_data['status'] == 'success':
              image = image_data['image']

              # Decode the base64 image data
              img_data = image.split(',')[1]
              img = Image.open(BytesIO(base64.b64decode(img_data)))

              # Define the image path
              img_path = os.path.join(folder_path, 'image1.png')

              # Save the image
              img.save(img_path)
              print(f"Image saved for user ID: 1")
          else:
              print(f"Error fetching image: {image_data['message']}")
      else:
          print(f"Error: Unable to reach the URL. Status code: {image_response.status_code}")

      # URL of the PHP script to get user IDs
      user_ids_url = "https://medcenter.com.ng/bvas/echo.php"

      # URL to fetch images for a specific user ID
      images_url_template = 'https://medcenter.com.ng/bvas/fetch_images.php?user_id={}'

      # Call the PHP script to get user IDs
      response = requests.get(user_ids_url)

      # Check if the request was successful
      if response.status_code == 200:
          # Parse the JSON response
          user_ids = response.json()
          print("Response from PHP script:", user_ids)

          # Base directory where folders will be created
          base_dir = "user_folders"

          # Create the base directory if it doesn't exist
          if not os.path.exists(base_dir):
              os.makedirs(base_dir)

          # Create a folder for each user ID and fetch/save images
          for user_id in user_ids:
              # Define the path for the new folder
              folder_path = os.path.join(base_dir, str(user_id))

              # Create the folder if it doesn't already exist
              if not os.path.exists(folder_path):
                  os.makedirs(folder_path)
                  print(f"Folder created for user ID: {user_id}")

                  # Fetch the images for the user ID
                  images_url = images_url_template.format(user_id)
                  images_response = requests.get(images_url)
                  images_data = images_response.json()

                  # Check if the response is successful
                  if images_data['status'] == 'success':
                      images = images_data['images']

                      # Save each image in the respective folder and prepare for pickle
                      saved_images = []
                      for i, img_data in enumerate(images):
                          # Decode the base64 image data
                          img_data = img_data.split(',')[1]
                          img = Image.open(BytesIO(base64.b64decode(img_data)))

                          # Define the image path
                          img_path = os.path.join(folder_path, f'image{i+1}.png')

                          # Save the image
                          img.save(img_path)
                          saved_images.append(img_path)
                          print(f"Image {i+1} saved for user ID: {user_id}")


                          # Directory containing folders with images
                          directory = '/content/gdrive/MyDrive/csefacerec/enrol/user_folders'

                          # Initialize a dictionary to store face encodings with corresponding names
                          face_encodings_dict = {}

                          # Iterate over each folder in the directory
                          for folder_name in os.listdir(directory):
                              folder_path = os.path.join(directory, folder_name)
                              if os.path.isdir(folder_path):
                                  # Iterate over each image file in the folder
                                  for filename in os.listdir(folder_path):
                                      image_path = os.path.join(folder_path, filename)
                                      # Load the image
                                      image = face_recognition.load_image_file(image_path)
                                      # Encode faces
                                      face_encodings = face_recognition.face_encodings(image)
                                      # Tag the face encodings with the folder name
                                      if folder_name not in face_encodings_dict:
                                          face_encodings_dict[folder_name] = []
                                      face_encodings_dict[folder_name].extend(face_encodings)

                          # Save the face encodings dictionary to a file
                          encodings_file = 'known_face_encodings.pkl'
                          with open(encodings_file, 'wb') as f:
                              pickle.dump(face_encodings_dict, f)

                          print("Face encodings saved to", encodings_file)

                      # Display the images
                      plt.figure(figsize=(15, 10))

                      for i, img_data in enumerate(images):
                          img_data = img_data.split(',')[1]
                          img = Image.open(BytesIO(base64.b64decode(img_data)))


                  else:
                      print(f"Error fetching images for user ID {user_id}: {images_data['message']}")
              else:
                  print(f"Folder already exists for user ID: {user_id}")
      else:
          print(f"Failed to retrieve data. HTTP Status code: {response.status_code}")

      encodings_file = '/content/gdrive/MyDrive/csefacerec/enrol/known_face_encodings.pkl'
      with open(encodings_file, 'rb') as f:
          known_face_encodings_dict = pickle.load(f)

      # Extract face encodings and corresponding names from the dictionary
      known_face_encodings = []
      known_face_names = []
      for name, encodings_list in known_face_encodings_dict.items():
          known_face_encodings.extend(encodings_list)
          known_face_names.extend([name] * len(encodings_list))

      image_path="/content/gdrive/MyDrive/csefacerec/enrol/voter/1/image1.png";
      image = face_recognition.load_image_file(image_path)
      face_locations = face_recognition.face_locations(image)
      if face_locations:
              print("Face detected.")
              face_encodings = face_recognition.face_encodings(image, face_locations)
              for face_encoding in face_encodings:
                  # Compare this encoding with those already known
                  matches = face_recognition.compare_faces(known_face_encodings, face_encoding, tolerance=0.2)
                  name = "Unknown"

                  # Find the index of the first match
                  if True in matches:
                      first_match_index = matches.index(True)
                      name = known_face_names[first_match_index]

                  print("Face recognized as:", name)
                  #root_ref.child('Students').child('open').set({'grant': 'yes',})
                  break
      else:
              print("No face detected. Retrying...")
              #root_ref.child('Students').child('open').set({'grant': 'no',})
              os.remove(image_path)

Streaming output truncated to the last 5000 lines.
Image saved for user ID: 1
Response from PHP script: ['1', '2', '3', '45', '7', '76', '85', '90', '97', 'VIN123', 'VIN243', 'VIN345', 'VIN769', 'VIN795 ', 'VN9034', 'vu237qIxRWbGNlF']
Folder already exists for user ID: 1
Folder already exists for user ID: 2
Folder already exists for user ID: 3
Folder already exists for user ID: 45
Folder already exists for user ID: 7
Folder already exists for user ID: 76
Folder already exists for user ID: 85
Folder already exists for user ID: 90
Folder already exists for user ID: 97
Folder already exists for user ID: VIN123
Folder already exists for user ID: VIN243
Folder already exists for user ID: VIN345
Folder already exists for user ID: VIN769
Folder already exists for user ID: VIN795 
Folder already exists for user ID: VN9034
Folder already exists for user ID: vu237qIxRWbGNlF
Face detected.
Face recognized as: Unknown
Directory 'verification' created.
Image saved for user ID: 1
Response from PHP s